# Choreography Pattern — Notes and Runbook

This file collects the relevant notes, run commands, and quick tests for the Choreography deployment in this repo.

## Purpose
- Choreography pattern uses Redis Pub/Sub and services that react to events.
- Services: `order-service`, `inventory-service`, `payment-service`, `shipping-service`.

## Compose / Ports
From `saga_pattern/choreography_pattern/compose.yaml`:
- MySQL: 3306
- Redis: 6379
- order-service: host port `8001` -> container `8001`
- inventory-service: host port `8002` -> container `8002`
- payment-service: host port `8003` -> container `8003`
- shipping-service: host port `8004` -> container `8004`

## Start / Stop
Start the choreography stack (build and detach):

```bash
cd saga_pattern/choreography_pattern
docker-compose up -d --build
```

Stop and remove containers:

```bash
docker-compose down
```

## Quick smoke tests (host machine)
- Health check (order service):

```bash
curl -v http://localhost:8001/health || curl -v http://localhost:8001/
```

- Create order (example):

```bash
curl -s -X POST http://localhost:8001/orders \
  -H 'Content-Type: application/json' \
  -d '{"customerId":"customer-001","bookId":"book-123","quantity":1}'
```

- If you get `500` responses, collect logs:

```bash
docker-compose logs --tail=200 --follow
# or per service
docker-compose logs order-service --tail=200
```

## Notes / Troubleshooting Checklist
- Ensure MySQL & Redis containers are healthy (check `docker-compose ps` and healthchecks).
- Ensure DB initialization SQL (`dbconf/init_scripts/init.sql`) ran successfully; check MySQL logs for errors.
- Verify environment variables for DB connection in services (e.g., `MYSQL_HOST`, `MYSQL_USER`).
- If `500` errors persist, run the container interactively and test inside:

```bash
docker-compose exec order-service /bin/sh
# then inside, try: curl http://inventory-service:8002/health
```

## Useful commands
- Build only:

```bash
docker-compose build --no-cache
```

- View service logs across stack:

```bash
docker-compose logs --tail=200
```

- Run db scripts manually (example):

```bash
docker-compose exec mysql mysql -ucloudmart_user -pcloudmart_pass cloudmart_saga < /docker-entrypoint-initdb.d/init.sql
```
